In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
import quaternion

In [8]:
# path = "../data/tst_cal_005.csv"
path = "data5_tst_cal_004_onArmArbitraryMotions.csv"
df = pd.read_csv(path)
df.head(0)

,IMU_TIMESTAMP(0)_0,IMU_GYROSCOPE(0)_0,IMU_GYROSCOPE(0)_1,IMU_GYROSCOPE(0)_2,IMU_GYROSCOPE(1)_0,IMU_GYROSCOPE(1)_1,IMU_GYROSCOPE(1)_2,IMU_ACCELEROMETER(0)_0,IMU_ACCELEROMETER(0)_1,IMU_ACCELEROMETER(0)_2,...,IMU_LINEAR_ACCELERATION(1)_1,IMU_LINEAR_ACCELERATION(1)_2,IMU_QUATERNION(0)_0,IMU_QUATERNION(0)_1,IMU_QUATERNION(0)_2,IMU_QUATERNION(0)_3,IMU_QUATERNION(1)_0,IMU_QUATERNION(1)_1,IMU_QUATERNION(1)_2,IMU_QUATERNION(1)_3


## Define equations

In [9]:
J       = sympy.MatrixSymbol('J', 4, 10)
phi     = sympy.MatrixSymbol('phi', 4, 1)
epsilon = sympy.MatrixSymbol('epsilon', 10, 1)

gauss_newton_delta = (J*J.transpose()) ** -1 * J * epsilon
gauss_newton_delta

(J*J.T)**(-1)*J*epsilon

In [10]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

error = omegaR.dot(j1.cross(j2))
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
derr_theta1

omega_x*(sin(rho_1)*cos(theta_1)*cos(theta_2) + sin(rho_2)*sin(theta_1)*sin(theta_2)) + omega_y*(-sin(theta_1)*sin(theta_2)*cos(rho_2) - cos(rho_1)*cos(theta_1)*cos(theta_2)) + omega_z*(-sin(rho_1)*sin(theta_2)*cos(rho_2)*cos(theta_1) + sin(rho_2)*sin(theta_2)*cos(rho_1)*cos(theta_1))

## Agorithm

In [11]:
th1_t = np.pi/2
rh1_t = np.pi/2
th2_t = np.pi/2
rh2_t = np.pi/2

RMS = 0
error_vector = []

Jacobian = None

for i,row in df.iterrows():
    ## Retrieve data from csv file
    omega1 = np.array([ row['IMU_GYROSCOPE(0)_0'], row['IMU_GYROSCOPE(0)_1'], row['IMU_GYROSCOPE(0)_2']] )
    omega2 = np.array([ row['IMU_GYROSCOPE(1)_0'], row['IMU_GYROSCOPE(1)_1'], row['IMU_GYROSCOPE(1)_2']] )
    q1  = np.quaternion(row['IMU_QUATERNION(0)_0'], row['IMU_QUATERNION(0)_1'], row['IMU_QUATERNION(0)_2'], row['IMU_QUATERNION(0)_3'])
    q2  = np.quaternion(row['IMU_QUATERNION(1)_0'], row['IMU_QUATERNION(1)_1'], row['IMU_QUATERNION(1)_2'], row['IMU_QUATERNION(1)_3'])
    ## Compute relative values
    qR = q1.inverse() * q2
    omegaR = quaternion.rotate_vectors(qR,omega2) - omega1
    ## Calculate error
    values = [
        (omegaX,omegaR[0]),(omegaY,omegaR[1]),(omegaZ,omegaR[2]),
        (theta1,th1_t),(rho1,rh1_t),(theta2,th2_t),(rho2,rh2_t)
    ]
    error_val = error.subs(values)
    error_vector.append(error_val)
    RMS += float(error_val*error_val)
    ## Calculate Jacobian
    if Jacobian is None:
        Jacobian = sympy.Matrix(J_col.subs(values))
    else:
        Jacobian = Jacobian.row_join(J_col.subs(values))
    break




RMS = np.sqrt(RMS)
Jacobian

Matrix([
[ 5.43940552023626],
[ 5.57278026094009],
[-5.43940552023626],
[-5.57278026094009]])

## Compute C code

In [12]:
from sympy.utilities.codegen import codegen
[(c_name, c_code), (h_name, c_header)] = codegen(('error', error), "C99", "error_fnc", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('gauss_newton', gauss_newton_delta), "C99", "gauss_newton", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('error_dt1', derr_theta1), "C99", "error_derivatives", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('error_dr1', derr_rho1),   "C99", "error_derivatives", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('error_dt2', derr_theta2), "C99", "error_derivatives", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('error_dr2', derr_rho2),   "C99", "error_derivatives", header=False, empty=False)

with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)


In [13]:
# Declare symbols
from sympy.abc import x,y,z
jx1,jy1,jz1 = sympy.symbols('j_x1 j_y1 j_z1')
jx2,jy2,jz2 = sympy.symbols('j_x2 j_y2 j_z2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')

# Rotation vectors
j1 = sympy.Matrix([jx1, jy1, jz1 ])
j2 = sympy.Matrix([jx2, jy2, jz2 ])

# Relative angular velocity
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

# Error and derivatives compute
error = omegaR.dot(j1.cross(j2))
# derr_theta1 = sympy.diff(error,theta1)
# derr_rho1   = sympy.diff(error,rho1)
# derr_theta2 = sympy.diff(error,theta2)
# derr_rho2   = sympy.diff(error,rho2)

# J_col = c4,1, [
#     derr_theta1, derr_rho1, derr_theta2, derr_rho2
# ])
# J_col
error

omega_x*(j_y1*j_z2 - j_y2*j_z1) + omega_y*(-j_x1*j_z2 + j_x2*j_z1) + omega_z*(j_x1*j_y2 - j_x2*j_y1)

## Generate C code for spherical transformations

In [17]:
from sympy.abc import x,y,z
# atan2(sqrt(vector[0]*vector[0]+vector[1]*vector[1]) , vector[2]);
sph_theta = sympy.atan2(sympy.sqrt(z**2 + y*y),x)
sph_rho   = sympy.atan2(y,z)

sph = sympy.Matrix(2,1, [
    sph_theta, sph_rho
])

from sympy.utilities.codegen import codegen
[(c_name, c_code), (h_name, c_header)] = codegen(('spherical', sph), "C99", "spherical", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

## Generate C code for error and derivatives

In [18]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')

j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

error = omegaR.dot(j1.cross(j2))
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])


[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_1', J_col), "C99", "spherical", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

j1 = sympy.Matrix([
        sympy.cos(theta1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.sin(theta1)*sympy.cos(rho1)
    ])
j2 = sympy.Matrix([
        sympy.cos(theta2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.sin(theta2)*sympy.cos(rho2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

error = omegaR.dot(j1.cross(j2))
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])

[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_2', J_col), "C99", "spherical", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

Matrix([
[omega_x*(sin(rho_1)*cos(theta_1)*cos(theta_2) + sin(rho_2)*sin(theta_1)*sin(theta_2)) + omega_y*(-sin(theta_1)*sin(theta_2)*cos(rho_2) - cos(rho_1)*cos(theta_1)*cos(theta_2)) + omega_z*(-sin(rho_1)*sin(theta_2)*cos(rho_2)*cos(theta_1) + sin(rho_2)*sin(theta_2)*cos(rho_1)*cos(theta_1))],
[                                                                                   omega_x*sin(theta_1)*cos(rho_1)*cos(theta_2) + omega_y*sin(rho_1)*sin(theta_1)*cos(theta_2) + omega_z*(-sin(rho_1)*sin(rho_2)*sin(theta_1)*sin(theta_2) - sin(theta_1)*sin(theta_2)*cos(rho_1)*cos(rho_2))],
[omega_x*(-sin(rho_1)*sin(theta_1)*sin(theta_2) - sin(rho_2)*cos(theta_1)*cos(theta_2)) + omega_y*(sin(theta_1)*sin(theta_2)*cos(rho_1) + cos(rho_2)*cos(theta_1)*cos(theta_2)) + omega_z*(-sin(rho_1)*sin(theta_1)*cos(rho_2)*cos(theta_2) + sin(rho_2)*sin(theta_1)*cos(rho_1)*cos(theta_2))],
[                                                                                   -omega_x*sin(theta_2)*cos(rho_2)*cos(the